In [ ]:
from utilities import ExecutionTime
import dateutil.parser
from influxdb_client import InfluxDBClient
import csv
from dateutil.parser import parse
import _pickle as pickle
import json
import pendulum
from rich.console import Console

In [ ]:
"""
Connect to InfluxDB 2.0.9 - query data
Configure credentials
"""

console = Console() # Create rich console
influx_cloud_url = 'http://192.168.1.87:8086/'
influx_cloud_token = 'f0psEpdC89AllAEweF4FS3d3GQzBw-EkHT4tOhOHh3U2f-pBLZkn8FfabsjsdKAWM2NwbQn7XxPvltB1U-xctQ=='
bucket = 'OPC'
org = 'home'

In [ ]:
"""
Query Influxdb v2.0.9 for data.
"""

with InfluxDBClient(url=influx_cloud_url, token=influx_cloud_token) as client:
    """
    Query data
    """
    _measurement = 'data'  # What to query for.
    #f'|> range(start: 2023-12-24T17:00:00Z, stop: 2023-12-24T22:00:00Z) ' \   # 11am - 4pm  12/24/2023
    query = f'from(bucket: "{bucket}") ' \
            f'|> range(start: -10m) ' \
            f'|> filter(fn: (r) => r._measurement == "{_measurement}")' \
            f'|> filter(fn: (r) => r["_field"] == "pressure")' \
            f'|> aggregateWindow(every: 3s, fn: last, createEmpty: false)' \
            f'|> yield (name: "last")'
    
    console.print(f'Querying from InfluxDB cloud: "{query}" ...')
    console.print()
    
    query_api = client.query_api()
    timer = ExecutionTime()  # Start the timer
    tables = query_api.query(query=query, org=org)
    console.print(f"Query was performed in {timer.human()}!!")

In [ ]:
console.print(tables)
console.print(tables.to_json())